Segmenting and Clustering Neighborhoods in Toronto

by Nathan Rathge

1. Blank notebook created in Watson Studio.

2. Scrape the wikipedia page using pandas.

In [1]:
# Find out the number of tables on the page
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

print(len(dfs))

3


In [2]:
# Try the first table.
df = dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
# Check the size of the dataframe
df.shape

(180, 3)

3. Clean the dataframe.

In [4]:
# Remove rows where Borough is "Not assigned"
# First replace "Not assigned" with NaN
import numpy as np
df["Borough"].replace("Not assigned", np.nan, inplace=True)
# Drop the rows with NaN values
df.dropna(subset=["Borough"], axis=0, inplace=True)
# Reset the index
df.reset_index(drop=True, inplace=True)
df.head(5)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
df.shape

(103, 3)

In [6]:
# Check if any neighborhoods need to be combined.
df['Postal Code'].value_counts()

M4Y    1
M4X    1
M5V    1
M9R    1
M7R    1
      ..
M1K    1
M4W    1
M9W    1
M1R    1
M9A    1
Name: Postal Code, Length: 103, dtype: int64

In [7]:
# Since the size of the value_counts dataframe is the same as the df, no combinations need to be made.

In [8]:
# Check for "Not assigned" in Neighborhood column.
NotA = df['Neighbourhood'].str.find("Not assigned")
NotA.value_counts()

-1    103
Name: Neighbourhood, dtype: int64

In [9]:
# Since value_counts dataframe does not find "Not assigned", no neighborhood combinations need to be made.

In [10]:
# Display the final shape of the dataframe.
df.shape

(103, 3)

This is the end of part one of the peer graded assignment.

This is the beginning of part two of the peer graded assignment.

Getting the GPS coordinates of each neighborhood.

In [11]:
# Download the data file and read it into a dataframe.
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
torontogps = pd.read_csv('Geospatial_Coordinates.csv')
torontogps.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Merge the neighborhood data with the postal code data.
dfgps=pd.merge(df,torontogps,on='Postal Code',how='outer')
dfgps.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


This is the end of part two of the peer graded assignment.

This is the beginning of part three of the peer graded assignment.

In [13]:
# Count the number of boroughs and neighborhoods in the Toronto data.
print('The dataframe has {} boroughs and {} neighborhoods.'.format( len(dfgps['Borough'].unique()), dfgps.shape[0]) )

The dataframe has 10 boroughs and 103 neighborhoods.


In [14]:
# Get the GPS coordinates of Toronto.
from geopy.geocoders import Nominatim
address = 'Toronto, ON, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [15]:
# Import Folium libraries.
!pip -q install folium
import folium
print('Libraries imported.')

Libraries imported.


In [16]:
# Create map of the Toronto area using latitude and longitude values.
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# Add markers to map.
for lat, lng, borough, neighborhood in zip(dfgps['Latitude'], dfgps['Longitude'], dfgps['Borough'], dfgps['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_toronto)
map_toronto

In [17]:
# Simplify the data to only include Downtown Toronto
toronto_data = dfgps[dfgps['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [18]:
# Get the shape of the data.  There are 19 neighbourhoods in Downtown Toronto.
toronto_data.shape

(19, 5)

In [61]:
# Create map of Downtown Toronto only using latitude and longitude values.  The same longitude and latitude are used for downtown.
map_downtoronto = folium.Map(location=[latitude, longitude], zoom_start=13)
# Add markers to map.
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7, parse_html=False).add_to(map_downtoronto)
map_downtoronto

In [20]:
# Define Foursquare credentials and version.
CLIENT_ID = 'hidden' # your Foursquare ID
CLIENT_SECRET = 'hidden' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: hidden
CLIENT_SECRET:hidden


In [21]:
# Define a radius and define a URL to obtain data from Foursquare.
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret= {}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=hidden&client_secret= hidden&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=100'

In [24]:
# Get venues within 500 meters of Downtown Toronto.
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f97868db8e5bb202e7996b7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 90,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

In [25]:
# Define a function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
# Create a dataframe for the venues in Downtown Toronto.
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues=nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis= 1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Japango,Sushi Restaurant,43.655268,-79.385165
4,Indigo,Bookstore,43.653515,-79.380696


In [46]:
nearby_venues.shape

(90, 4)

In [47]:
# Create a function to repeat the same process to all the neighborhoods in Downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for namey, lat, lng in zip(names, latitudes, longitudes):
        print(namey)
        # create the API request URL
        url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client _secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
        
        # make the GET request
        results2 = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            namey,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results2])

    nearby_venues2 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues2.columns = ['Neighbourhood',
            'Neighborhood Latitude',
            'Neighborhood Longitude',
            'Venue',
            'Venue Latitude',
            'Venue Longitude',
            'Venue Category']
    return(nearby_venues2)

In [48]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],latitudes=toronto_data['Latitude'],longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [49]:
print(toronto_venues.shape)
toronto_venues.head()

(1710, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,"Regent Park, Harbourfront",43.65426,-79.360636,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,"Regent Park, Harbourfront",43.65426,-79.360636,Eggspectation Bell Trinity Square,43.653144,-79.381980,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Japango,43.655268,-79.385165,Sushi Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Indigo,43.653515,-79.380696,Bookstore


In [50]:
toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,90,90,90,90,90,90
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",90,90,90,90,90,90
Central Bay Street,90,90,90,90,90,90
Christie,90,90,90,90,90,90
Church and Wellesley,90,90,90,90,90,90
"Commerce Court, Victoria Hotel",90,90,90,90,90,90
"First Canadian Place, Underground city",90,90,90,90,90,90
"Garden District, Ryerson",90,90,90,90,90,90
"Harbourfront East, Union Station, Toronto Islands",90,90,90,90,90,90


In [51]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 63 unique categories.


In [57]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[: -1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()
toronto_onehot.to_csv('toronto.csv')

In [58]:
toronto_onehot.shape

(1710, 64)

In [59]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,American Restaurant,Art Museum,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,...,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Women's Store
0,Berczy Park,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
2,Central Bay Street,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
3,Christie,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
4,Church and Wellesley,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
5,"Commerce Court, Victoria Hotel",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
6,"First Canadian Place, Underground city",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
7,"Garden District, Ryerson",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
8,"Harbourfront East, Union Station, Toronto Islands",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
9,"Kensington Market, Chinatown, Grange Park",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111


In [60]:
toronto_grouped.shape

(19, 64)

In [43]:
num_top_venues = 5
for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Christie----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Church and Wellesley----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Commerce Court, Victoria Hot

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.011111,Coffee Shop,Clothing Store,Hotel,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office,Plaza,Restaurant
1,0.011111,Coffee Shop,Clothing Store,Hotel,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office,Plaza,Restaurant
2,0.011111,Coffee Shop,Clothing Store,Hotel,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office,Plaza,Restaurant
3,0.011111,Coffee Shop,Clothing Store,Hotel,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office,Plaza,Restaurant
4,0.011111,Coffee Shop,Clothing Store,Hotel,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office,Plaza,Restaurant


In [156]:
neighborhoods_venues_sorted.shape

(19, 11)

In [78]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [74]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [71]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

KeyError: 'Cluster Labels'

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

KeyError: 'Cluster Labels'